In [1]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc =SparkContext()
sqlContext = SQLContext(sc)
from nltk.corpus import stopwords

In [2]:
stop_word_list = stopwords.words('english')
#  to quickly test if a word is not a stop word, use a set:
stop_word_set = set(stop_word_list)
stop_word_set = list(stop_word_set)
data = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('articles-train.csv')
drop_list = ['Dates', 'Topic', 'Page']
data = data.select([column for column in data.columns if column not in drop_list])
data.show(5)


+--------+--------------------+
|Category|                Body|
+--------+--------------------+
|business|SEOUL  WITH A FAL...|
|business|JEFFERSON CITY  M...|
|business|WASHINGTON  THE T...|
|business|REUTERS    METLIF...|
|  sports|DALLAS  WHEN DALL...|
+--------+--------------------+
only showing top 5 rows



In [3]:
data.printSchema()

root
 |-- Category: string (nullable = true)
 |-- Body: string (nullable = true)



# Top 20 crime categories:

In [4]:
from pyspark.sql.functions import col
data.groupBy("Category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+-------------+-----+
|     Category|count|
+-------------+-----+
|     business| 2309|
|     politics| 1666|
|       sports|  571|
|entertainment|  464|
+-------------+-----+



In [5]:
data.groupBy("Body") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-----+
|                Body|count|
+--------------------+-----+
|HERE IS THE APRIL...|    9|
|BEVERLY HILLS  CA...|    9|
|WANT TO GET THIS ...|    8|
|BEIJING  U S  TRE...|    8|
|JOANNE KIM AND AY...|    7|
|WANT TO GET THIS ...|    7|
|INDIANAPOLIS  IND...|    7|
|WANT TO GET THIS ...|    7|
|WASHINGTON  FPI M...|    7|
|INDIANAPOLIS  OKL...|    7|
|LONDON  A RESURGE...|    7|
|AUBURN  WASH   PR...|    6|
|IT FEELS LIKE KAN...|    6|
|PHOENIX  WHEN FOR...|    6|
|INDIANAPOLIS  IND...|    6|
|LONDON  BRITAIN S...|    6|
|WE MAY NEVER STOP...|    6|
|AT THE 2014 CONSU...|    6|
|HERE ARE THE WEEK...|    6|
|VENTURA  CALIF   ...|    6|
+--------------------+-----+
only showing top 20 rows



In [6]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
# stop words
# add_stopwords = ["http","https","amp","rt","t","c","the"] 
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(stop_word_set)
# bag of words count
# countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
# label_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")
# pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
# # Fit the pipeline to training documents.
# pipelineFit = pipeline.fit(data)
# dataset = pipelineFit.transform(data)
# dataset.show(5)

In [8]:
# set seed for reproducibility
# (trainingData, testData) = dataset.randomSplit([0.8, 0.2], seed = 100)
# print("Training Dataset Count: " + str(trainingData.count()))
# print("Test Dataset Count: " + str(testData.count()))

In [9]:
from pyspark.ml.feature import HashingTF, IDF
label_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, hashingTF, idf, label_stringIdx])
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)
(trainingData, testData) = dataset.randomSplit([0.8, 0.2], seed = 324)
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("Body","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 20, truncate = 30)

+------------------------------+-------------+------------------------------+-----+----------+
|                          Body|     Category|                   probability|label|prediction|
+------------------------------+-------------+------------------------------+-----+----------+
|DALIAN MANILA  CHINA PLANS ...|     business|[0.9993406144602194,1.38312...|  0.0|       0.0|
|LONDON FRANKFURT  GERMANY S...|     business|[0.9980119859619365,4.94516...|  0.0|       0.0|
|SYDNEY  AUSTRALIA S BIGGEST...|     business|[0.997918087143643,0.001395...|  0.0|       0.0|
|PARIS  SOCGEN S CHIEF EXECU...|     business|[0.997250963812292,0.001054...|  0.0|       0.0|
|ADEN  YEMEN  THE YOUNG MOTH...|     business|[0.9964283193746383,8.25149...|  0.0|       0.0|
|ADEN  YEMEN  THE YOUNG MOTH...|     business|[0.9964283193746383,8.25149...|  0.0|       0.0|
|HONG KONG  A GADGET MAKER  ...|     business|[0.9952530049599432,0.00187...|  0.0|       0.0|
|HONG KONG  A GADGET MAKER  ...|     business|[0.9

In [10]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.6566287185364578

In [20]:
test = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('articles-test.csv')
drop_list = ['Dates', 'Topic', 'Page']
test = data.select([column for column in data.columns if column not in drop_list])
test.show(5)

+-------------+--------------------+
|     Category|                Body|
+-------------+--------------------+
|     business|CAIRO  EGYPT WILL...|
|     business|WANT TO GET THIS ...|
|     politics|WASHINGTON  U S  ...|
|     business|WITH THE ARRIVAL ...|
|entertainment|ON THURSDAY  SOON...|
+-------------+--------------------+
only showing top 5 rows



In [21]:
from pyspark.sql.functions import col
test.groupBy("Category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+-------------+-----+
|     Category|count|
+-------------+-----+
|     business|  435|
|     politics|  352|
|       sports|  123|
|entertainment|   90|
+-------------+-----+



In [22]:
label_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, hashingTF, idf, label_stringIdx])
pipelineFit = pipeline.fit(test)

In [23]:
testset = pipelineFit.transform(test)

In [24]:
predictions = lrModel.transform(testset)
predictions.filter(predictions['prediction'] == 0) \
    .select("Body","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 20, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                          Body|Category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|LONDON  LLOYD S OF LONDON  ...|business|[0.9987644291294391,5.51420...|  0.0|       0.0|
|NEW YORK  BILLIONAIRE INVES...|politics|[0.9971454845961765,8.76934...|  1.0|       0.0|
|NEW YORK  BILLIONAIRE INVES...|business|[0.9971454845961765,8.76934...|  0.0|       0.0|
|ATHENS PATRAS  GREECE  GREE...|business|[0.9956880840988583,0.00165...|  0.0|       0.0|
|LPC    THE SIZE OF FUND FIN...|business|[0.9938978451651596,0.00168...|  0.0|       0.0|
|SAN FRANCISCO  MOST BIG BAN...|business|[0.9932966005974454,0.00251...|  0.0|       0.0|
|LOS ANGELES LONDON  SWISS B...|business|[0.9931797480241308,0.00275...|  0.0|       0.0|
|LONDON  A RESURGENT DOLLAR ...|business|[0.991502433831096,0.006527...|  0.0|       0.0|
|FRANKFURT

In [25]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.6517946810259405

In [27]:
#Naive Bayes
